Here, we are exploring the data going into my neural network trader


In [17]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from dotenv import load_dotenv
from sklearn.preprocessing import MinMaxScaler
import numpy as np 
import pandas as pd 
import os
import datetime

In [4]:
dotenv_path = r'C:\Users\srerr\Documents\Projects\PersonalProjects\stonks\alpaca\.env'
load_dotenv(dotenv_path = dotenv_path)
api_key = os.environ['ALPACA_PAPER_TRADING_KEY_ID']
secret_key = os.environ['ALPACA_PAPER_TRADING_SECRET_KEY']
trading_client = TradingClient(api_key = api_key, secret_key=secret_key)

In [54]:
crypto_historical_data_client = CryptoHistoricalDataClient()
start_time = datetime.datetime.utcnow() - datetime.timedelta(minutes = 1000)
bars_request = CryptoBarsRequest(
    symbol_or_symbols='ETH/USD', 
    start = start_time, 
    timeframe=TimeFrame.Minute
)

df = crypto_historical_data_client.get_crypto_bars(bars_request).df
df.drop_duplicates(inplace=True)

In [55]:
close_prices = df['close'].values

In [56]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_data = scaler.fit_transform(X = close_prices.reshape(-1, 1))

I want to understand the structure of the input data into this model

In [57]:
def get_train_data(scaled_data: np.array, look_back: int = 100): 
    x, y = [], []
    for i in range(look_back, len(scaled_data)):
        x.append(scaled_data[i - look_back:i, :])
        y.append(scaled_data[i, :])

    return np.array(x), np.array(y)

In [58]:
x_train, y_train = get_train_data(scaled_data)

In [59]:
x_train.shape

(884, 100, 1)

In [60]:
y_train.shape

(884, 1)

In [61]:
x_train[0]

array([[-0.35951662],
       [-0.3836858 ],
       [-0.3776435 ],
       [-0.37160121],
       [-0.34138973],
       [-0.34138973],
       [-0.33534743],
       [-0.33534743],
       [-0.32326284],
       [-0.28700906],
       [-0.30513595],
       [-0.31117825],
       [-0.31117825],
       [-0.29305136],
       [-0.28700906],
       [-0.29909366],
       [-0.29305136],
       [-0.25075529],
       [-0.25679758],
       [-0.25075529],
       [-0.23867069],
       [-0.25075529],
       [-0.26283988],
       [-0.28096677],
       [-0.28096677],
       [-0.32326284],
       [-0.32326284],
       [-0.34743202],
       [-0.34743202],
       [-0.34743202],
       [-0.35951662],
       [-0.35951662],
       [-0.35951662],
       [-0.35951662],
       [-0.35951662],
       [-0.35951662],
       [-0.35951662],
       [-0.35951662],
       [-0.32326284],
       [-0.32326284],
       [-0.32326284],
       [-0.32326284],
       [-0.31722054],
       [-0.31722054],
       [-0.31722054],
       [-0

In [62]:
y_train[0]

array([-0.35347432])

So the model is taking as inputs a time series of the historical 100-period prices and trying to predict the next price. We do this rolling each time. 

In [64]:
x_train.shape[1]

100

In [65]:
x_train.shape[2]

1